## Импорт и определение библиотек

In [4]:
# отключение параллелизма
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [5]:
# ставим разные пакеты оттуда
!pip install langchain langchain-community langchain_huggingface unstructured pypandoc -q
!pip install faiss-cpu -q
!pip install langchain-ollama -q

In [6]:
# импортируем библиотеки
from langchain.document_loaders import WebBaseLoader, PyPDFLoader, UnstructuredEPubLoader
from langchain_core.documents import Document
from langchain.prompts import PromptTemplate
from langchain.tools import Tool
from langchain.agents import initialize_agent
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFaceEmbeddings
import tqdm as notebook_tqdm
from langchain_community.vectorstores import FAISS
from langchain.llms import Ollama
from langchain_ollama import OllamaLLM
from langchain.text_splitter import TextSplitter
from typing import List

USER_AGENT environment variable not set, consider setting it to identify your requests.


# Готовим коктейль с помощью RAG
## Чутка `embeddings` с `HuggingFace` для оснастки

In [7]:
# Чтение токена из файла
def read_token(file_path): 
    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith("HUGGINGFACEHUB_API_TOKEN"):
                return line.split('=')[1].strip()
    raise ValueError("Token not found in the specified file.")

# Загрузка токена
#hf_token = read_token('../utils.key')

# Формирование эмбеддингов
hf_embeddings_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",  
    #"cointegrated/LaBSE-en-ru", 
    model_kwargs={"device": "cpu"} 
)

/home/elijah/Documents/LLM/cocktail_qr/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Можно пропустить (если используете наполненную чашу векторного хранилища)
### Немного `Document Loader`

In [45]:
# Загрузка EPUB файла от бармена-литературоведа
loader = UnstructuredEPubLoader('../docs/Federle.epub', show_progress=True)
doc_epub_mistral = loader.load()

# смотрим, скока получилось загрузить
print('pages count:', len(doc_epub_mistral))
print('max chars on page:', max([len(page.page_content) for page in doc_epub_mistral]))

pages count: 1
max chars on page: 107261


In [ ]:
# Загружаем книгу от крутого бармена
#loader = WebBaseLoader(url)
"""loader = PyPDFLoader('../docs/Bortnik_1000.pdf')
data = loader.load()

print('pages count:', len(data))
print('max chars on page:', max([len(page.page_content) for page in data]))"""

# на тот случай, если решим загружать подобную литературу пачкой pdf
"""
#!pip install unstructured "unstructured[pdf]" -q
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader("../docs", glob="**/*.pdf", show_progress=True)
data = loader.load()"""

### Исполним `Text splitters` и разобьём на чанки

In [46]:
class RecipeSplitter(TextSplitter):
    def __init__(self):
        super().__init__(chunk_size=1000, chunk_overlap=200)
    
    def split_text(self, text: str) -> List[str]:
        # Убираем все пустые строки и лишние пробелы
        cleaned_text = "\n".join([line.strip() for line in text.split("\n") if line.strip()])
        
        # Разделяем текст на рецепты по маркеру "НАЗВАНИЕ_РЕЦЕПТА\nАВТОР"
        chunks = []
        current_chunk = []
        lines = cleaned_text.split("\n")
        
        i = 0
        while i < len(lines):
            line = lines[i]
            # Проверяем начало нового рецепта
            if i + 1 < len(lines) and lines[i+1].isupper() and "(" in lines[i+1] and ")" in lines[i+1]:
                if current_chunk:
                    chunks.append("\n".join(current_chunk))
                    current_chunk = []
                current_chunk.append(line)
                current_chunk.append(lines[i+1])
                i += 2
                continue
            
            current_chunk.append(line)
            i += 1
        
        if current_chunk:
            chunks.append("\n".join(current_chunk))
        
        return chunks
    
splitter = RecipeSplitter()

In [47]:
split_documents = []
for page in doc_epub_mistral:
    split_documents += splitter.create_documents([page.page_content], metadatas=[page.metadata])

print('Количество чанков:', len(split_documents))

Количество чанков: 73


### Наполним чашу `Vector Store`

In [50]:
# Если нужно сохранить новый документ в базу
try:
    db_embed = FAISS.from_documents(
            split_documents, 
            hf_embeddings_model
        )
except Exception as e:
    raise RuntimeError(f"Ошибка при создании базы данных FAISS: {e}")


# Сохраняем векторную базу локально
db_embed.save_local("../data/faiss_db_epub_mistral")

## Нужно запустить (если используете готовое векторное хранилище)
### Испьём чашу `Vector Store`

In [51]:
# Если нужно изъять из уже сохранённого документа из базы
# Загрузка векторной базы данных из локального файла
db_embed = FAISS.load_local(
        "../data/faiss_db_epub_mistral", 
        hf_embeddings_model,
        allow_dangerous_deserialization = True
    )

In [52]:
query = "грейпфрутовый сок, ржаной виски"
results = db_embed.similarity_search(query, k=5)
for result in results:
    print(result.page_content)

Лайм для старого морехода
СКАЗАНИЕ О СТАРОМ МОРЕХОДЕ (1798)
СЭМЮЭЛ ТЕЙЛОР КОЛЬРИДЖ
Когда в следующий раз застрянете на необитаемом острове, не поддавайтесь искушению возопить: «Вода, вода, одна вода, мы ничего не пьем…»[2] Вашим товарищам по несчастью неинтересны литературные цитаты – им нужны коктейли, чтобы утолить жажду, и психотерапевт, чтобы усмирить душевные метания. Кстати, вышеупомянутая строка взята из эпической поэмы о коварствах стихий, гневливых океанских водах и дохлых птицах: да, уважаемый альбатрос, незачем было увязываться за этой посудиной! (Мораль всей истории: руки прочь от божьих тварей, шкипер!) Отпразднуем же наше пребывание на суше свежестью лайма, добавленной к канонической солености морских вод (и, пожалуй, останемся лучше на пляже):
60 мл сока лайма;
60 мл грейпфрутового сока;
50 мл джина;
морская соль для красты.
Украшаем охлажденный стакан хайбол крастой из морской соли. Помещаем в него лед, наливаем ингредиенты и размешиваем методом стир, то есть ложкой. А 

# Месье системный промт

In [53]:
system_prompt = """
        Ты — помощник для поиска рецептов коктейлей. У тебя есть доступ к базе данных с рецептами, вдохновлёнными литературными произведениями.
        Когда пользователь вводит ингредиенты, используй инструмент "Cocktail Recipe Finder", чтобы найти подходящие рецепты.
        Если рецепт не найден, предложи альтернативные варианты или уточни запрос.

        ===
        
        Пример:
        Пользователь: Какие коктейли можно сделать из водки и апельсинового сока?
        Агент: Использую инструмент "Cocktail Recipe Finder" для поиска рецептов. Вот что найдено: [рецепт].

        ===

        Пример корректного ответа по введённым ингредиентам (ржаной виски, грейпфрутовый сок):
        Коктейль "Рожь и предубеждение"
        Состав:
        - Ржаной виски (50 мл)
        - Грейпфрутовый сок (90 мл)
        Способ приготовления:
        - Ингредиенты выливаем в стакан рокс поверх кубиков льда
    """

In [54]:
# Промпт для LangChain
qa_prompt = PromptTemplate(
    template=system_prompt + "\nКонтекст: {context}\nВопрос: {question}\nОтвет:",
    input_variables=["style", "context", "question"]
)

## Полирнём `Retriver` и инициализирует `llm` для вкуса

In [69]:
# Инициализация модели
llm = OllamaLLM(
    model="mistral:instruct",
    temperature=0.2,          # Для креативности
    num_predict=512,         # Увеличенная длина ответа
    stop=["\\n\\n", "###"],   # Стоп-токены
    top_k=40
)

In [79]:
# Модифицируем параметры ретривера
retriever = db_embed.as_retriever(
    search_type="mmr",              # Maximal Marginal Relevance
    search_kwargs={
        "k": 5,                     # Уменьшаем количество возвращаемых документов
        "fetch_k": 20,              # Уменьшаем общий пул для поиска
        #"lambda_mult": 0.5          # Баланс между релевантностью и разнообразием
    }
)

# Инициализация цепочки RetrievalQA
retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,  # Ваш ретривер (например, FAISS)
    #chain_type_kwargs={"prompt": qa_prompt},
    return_source_documents=True
)

# Миксуем рецепт коктейля `agent` 00х

In [80]:
# Создание инструмента
agent_tools = [
    Tool(
        name="Cocktail Recipe Finder",
        func=retrieval_qa.run,
        description="Используй этот инструмент, чтобы найти рецепты коктейлей по заданным ингредиентам. Вводи список ингредиентов, и инструмент вернёт подходящие рецепты. Пример: ['сок', 'виски']."
    )
]

# создание агента
agent = initialize_agent(
    tools=agent_tools, 
    llm=llm, 
    agent="conversational-react-description",       #"zero-shot-react-description", "plan-and-execute"
    verbose=True,
    system_prompt=system_prompt                     # не забыть указать системный промт для вкуса
)

# Задаём общие параметры, после чего коктейль готов
## Вначале коктейль

In [72]:
# Объявление функции на ввод данных от пользователя
def get_user_input():
    """
    Запрашивает у пользователя ингредиенты для поиска рецептов.
    """
    user_input = input("Введите ингредиенты, разделенные запятыми: ")
    ingredients = [ingredient.strip() for ingredient in user_input.split(",")]
    return ingredients


# Обработка ответа
def format_response(response):
    if "не знаю" in response.lower() or "не найдено" in response.lower():
        return "К сожалению, я не нашёл коктейлей с этими ингредиентами. Попробуйте другие ингредиенты."
    else:
        return response
    

# Функция выбора стиля ответа
def choose_style(styles):
    chosen_style = input("Выберите номер стиля (по умолчанию 4): ") or "4"
    return styles.get(chosen_style, styles["4"])#["name"]

In [82]:
def main():
    # Определение стилей
    styles = {
        "1": "стиль космического ужаса и хтонического мрака Говарда Ф. Лавкрафта",
        "2": "гопническо-быдляцкий жаргон",
        "3": "экспериментальный стиль нарезок Уильяма Берроуза",
        "4": "стиль без изменений"  # Стиль по умолчанию
    }
    
    while True:
        ingredients = get_user_input()
        if not ingredients:
            print("Вы не ввели ингредиенты. Попробуйте снова.")
            continue
        
        # Преобразуем массив ингредиентов в строку
        ingredients_str = ", ".join(ingredients)
        
        # Формируем текстовый запрос
        user_prompt = (
            """Напиши, какие коктейли можно изготовить из представленных ниже ингредиентов. 
            Для каждого коктейля опиши подробный способ приготовления и укажи пропорции. 
            Отвечай строго на русском языке, используя чёткий и лаконичный формат. 
            Не добавляй лишнюю информацию, которая не относится к рецепту. 
            Ингредиенты: {ingredients}""".format(ingredients=ingredients_str)
        )
        response = retrieval_qa.invoke({"query": user_prompt})  
        
        # Получаем ответ без вывода его на экран
        original_response = response["result"]
        # Выбор стиля
        style = choose_style(styles)
        prompt = PromptTemplate(input_variables=['output_text', 'style'],
                                template='''Добро пожаловать в поиск коктейлей!\nПерепиши этот текст в заданном стиле: {output_text}\nСтиль: {style}.\nРезультат:\n''')
        
        # Генерация текста с использованием оригинального ответа и выбранного стиля
        styled_prompt = prompt.format(output_text=original_response, style=style)
        
        # Применение выбранного стиля к оригинальному ответу
        styled_response = llm.invoke(styled_prompt)
        print(styled_response)
        break
    
    return styled_response

if __name__ == "__main__":
    # Сохраняем результат в переменную
    recipe = main()

1. Коктейль "Грейпфрутовый Скорретто" (Grapefruit Sour):
     - Взбалтываем 60 мл грейпфрутового сока, 30 мл ржаного виски и 15 мл сиропа в чаше с льдом, заполненной духами древних тайн.
     - Выливаем смесь в стакан коллинз и добавляем 30 мл лимонного сока, возможно, приносящего нам неизведанные знания.
     - Украшаем листом грейпфрута или ломтиком ананаса, если вы хотите добавить вкус земли к своему коктейлю из глубин.

  2. Коктейль "Ржаный Виски на льду" (Rye on the Rocks):
     - В стакан для коктейлей добавляем 60 мл ржаного виски и несколько кубиков льда, заполненных неизведанными силами.
     - Необязательно, но можно дополнить ломтиком айсберга или листом лайма, если вы хотите добавить вкус моря к своему коктейлю из глубин.


# Реюнион

In [1]:
from langchain.text_splitter import TextSplitter
from typing import List
    
class RecipeSplitter(TextSplitter):
    def __init__(self):
        super().__init__(chunk_size=1000, chunk_overlap=0)
    
    def split_text(self, text: str) -> List[str]:
        # Убираем все пустые строки и лишние пробелы
        cleaned_text = "\n".join([line.strip() for line in text.split("\n") if line.strip()])
        
        # Разделяем текст на рецепты по маркеру "НАЗВАНИЕ_РЕЦЕПТА\nАВТОР"
        chunks = []
        current_chunk = []
        lines = cleaned_text.split("\n")
        
        i = 0
        while i < len(lines):
            line = lines[i]
            # Проверяем начало нового рецепта
            if i + 1 < len(lines) and lines[i+1].isupper() and "(" in lines[i+1] and ")" in lines[i+1]:
                if current_chunk:
                    chunks.append("\n".join(current_chunk))
                    current_chunk = []
                current_chunk.append(line)
                current_chunk.append(lines[i+1])
                i += 2
                continue
            
            current_chunk.append(line)
            i += 1
        
        if current_chunk:
            chunks.append("\n".join(current_chunk))
        
        return chunks

In [ ]:
# Пример текста
text = """
Рожь и предубеждение
ГОРДОСТЬ И ПРЕДУБЕЖДЕНИЕ (1813)
ДЖЕЙН ОСТИН
Роман благородных нравов XIX века подарил Джейн Остин шанс еще при жизни (хоть и недолгой – всего 41 год) вкусить свои пять минут славы. «Гордость и предубеждение» повествует о попытках семейства Беннет выдать замуж своих дочерей. От лица одной из пяти девиц и ведется рассказ. Увы, Элизабет – в одной из экранизаций блеснувшая выразительными скулами Киры Найтли – та еще язва, и ее жажда припечатать всех и каждого чуть ли не затмевает ее трепетное чувство к мистеру Дарси, джентльмену с завидным самомнением (зато ну очень богатому!). Впрочем, все кончается восхитительной двойной свадьбой. Вот и мы сосватаем две сильные натуры – терпкую пряность ржи и жизнеутверждающую горечь грейпфрута – и на славу погуляем на нашем импровизированном торжестве:


90 мл грейпфрутового сока;

50 мл ржаного виски.


Ингредиенты выливаем в стакан рокс поверх кубиков льда, размешиваем в ритме «сердце вскачь». Хотим подчеркнуть, юные леди, мы нисколечко не предубеждены против замужества, а лишь призываем вас уяснить главное: чтобы почувствовать себя королевой, не нужно и дворца (как, впрочем, и короля).


Кофейный ликер без меры
ЛЮБОВЬ ВО ВРЕМЯ ХОЛЕРЫ (1985)
ГАБРИЭЛЬ ГАРСИА МАРКЕС
Зачем вы, девочки, докторов заграничных любите? Отдашь такому сердце – и полвека промаешься, прежде чем найти то самое, что доктор прописал. Вот она какая, романтика по Маркесу: юноша и девушка загорелись друг к другу пылкой страстью, но девица возьми да выбери себе в мужья доктора – а возлюбленному от ворот поворот дала, крути, мол, в отместку с кем хочешь. Но над истинной страстью время не властно! И однажды, «пятьдесят один год, девять месяцев и четыре дня спустя» (надо же было высчитать!), после кончины Супруга-Номер-Один, те двое воссоединяются. Жемчужина колумбийской литературы, роман Гарсиа Маркеса заслуживает нашего салюта – сладкого, как истинная любовь, и пряного, как неувядающая страсть:


30 мл светлого рома;

15 мл кофейного ликера (например, «Калуа»);

60 мл нежирных сливок;

щепотка молотой корицы или мускатного ореха.


В стакан рокс укладываем лед и поливаем его ромом и кофейным ликером. Добавляем сливки и приправу по вкусу. А теперь опрокидываем нашу гремучую смесь во славу истинной страсти – даже если заморского врача вы видели только в телевизоре.
"""

# Инициализация сплиттера
splitter = RecipeSplitter()

# Разбиение текста на чанки
chunks = splitter.split_text(text)
for i, chunk in enumerate(chunks, 1):
    print(f"Чанк {i}:\n{chunk}\n{'-' * 40}")

In [13]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

# Инициализация эмбеддингов
hf_embeddings_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2", #"cointegrated/LaBSE-en-ru",
    model_kwargs={"device": "cpu"}
)

from langchain_core.documents import Document

# Преобразование строк в объекты Document
documents = [Document(page_content=chunk) for chunk in chunks]

# Создание векторного хранилища
try:
    db_embed = FAISS.from_documents(
        documents, 
        hf_embeddings_model
    )
except Exception as e:
    raise RuntimeError(f"Ошибка при создании базы данных FAISS: {e}")

# Сохранение векторной базы локально
db_embed.save_local("../data/faiss_db_epub_mistral")

query = "ржаной виски, грейпфрутовый сок"
results = db_embed.similarity_search(query, k=1)  # Ищем 1 наиболее релевантный чанк
for result in results:
    print(result.page_content)